In [1]:
from requests import get
from bs4 import BeautifulSoup
import pandas as pd
import re
from re import sub
import numpy as np

#data lists to append

names           = []
years           = []
imdb_ratings    = []
metascores      = []
votes           = []
runtimes        = []
gross           = []
certificates    = []
genres          = []

urls = []
pages = 1
url = 'https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&genres=horror&start='
print(urls)
#response = get(url)
#soup = BeautifulSoup(response.text, 'lxml')
for i in range(20):
    urls.append(url+str(pages))
    pages+=50
    print(urls[i])

#looping through containers
for i in urls:
    response = get(i)
    soup = BeautifulSoup(response.text, 'lxml')  
    movie_containers = soup.find_all('div', class_= 'lister-item mode-advanced')

    # Extract data from individual movie container
    for container in movie_containers:
        count = 0
        count0 = 0
        count1 = 0

        if container.find('div', class_ = 'ratings-imdb-rating') is not None:



            # The name of the movie
            name = container.h3.a.text
            names.append(name)
            # The year of the movie
            year = container.h3.find('span', class_ = 'lister-item-year').text
            n_year = re.findall(r'\d+', year)
            year = n_year[0]
            years.append(int(year))
            # The IMDB rating
            imdb = container.strong.text
            imdb_ratings.append(float(imdb))
            # The Metascore
            if container.find('span', class_ = 'metascore') is not None:
                m_score = container.find('span', class_ = 'metascore').text
                metascores.append(int(m_score))
                count0+=1
            else:
                count = 0
            # The runtime of the movie
            runtime = container.find('span', class_ = 'runtime').text
            n_runtime = re.findall(r'\d+', runtime)
            runtime = n_runtime[0]
            runtimes.append(int(runtime))
            # Certificates
            if container.p.find('span', 'certificate') is not None:
                certificate = container.p.find('span', 'certificate').get_text(strip=True)
                certificates.append(certificate)
            else:
                certificates.append(None)

            # The genre:
            if container.p.find('span', 'genre') is not None:
                genre = container.p.find('span', 'genre').get_text(strip=True)
                genres.append(genre)
            else:
                genres.append(None)
            #Gross and Votes of the Movie
            for i in container.find_all("span", attrs = {'name':'nv'}):
                str = i.get_text()
                #Checking for Gross or Votes Tags because they are identical
                if "$" in str:
                    count+=1
                    val = float(sub(r'[^\d.]', '', str))*1000000
                    gross.append(val)
                else:
                    vote = int(str.replace(',',''))
                    votes.append(vote)
                    count1+=1
            #Checking for NaN values
            if count == 1 and count0 == 1 and count1 == 1:
                continue
            elif count == 1 and count0 == 1 and count1 == 0:
                votes.append(np.nan)
            elif count == 1 and count0 == 0 and count1 == 1:
                metascores.append(np.nan)
            elif count == 1 and count0 == 0 and count1 == 0:
                metascores.append(np.nan)
                votes.append(np.nan)
            elif count == 0 and count0 == 1 and count1 == 1:
                gross.append(np.nan)
            elif count == 0 and count0 == 1 and count1 == 0:
                votes.append(np.nan)
                gross.append(np.nan)
            elif count == 0 and count0 == 0 and count1 == 1:
                gross.append(np.nan)
                metascores.append(np.nan)
            else:
                gross.append(np.nan)
                votes.append(np.nan)
                metascores.append(np.nan)

#Creating Dataframe from scraping data
test_df = pd.DataFrame(data = {
    'horror movie': names,
    'year': years,
    'imdb-ratings': imdb_ratings,
    'metascore': metascores,
    'votes': votes,
    'runtime' : runtimes,
    'gross*10^6' : gross,
    'certificates' : certificates

    #'genres' : genres
})
#print(test_df.info())
#test_df.replace(np.nan,'NULL')
#test_df.loc[test_df['metascore']== np.nan,'metascore'] = 'NULL'
#test_df.loc[test_df['gross*10^6']== np.nan,'gross*10^6'] = 'NULL'
test_df['metascore'].fillna("Null", inplace = True) 
test_df['gross*10^6'].fillna("Null", inplace = True) 
test_df['certificates'].fillna("Null", inplace = True)

test_df.to_csv("horror_movies.csv", sep=",", encoding="utf-8")
test_df

[]
https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&genres=horror&start=1
https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&genres=horror&start=51
https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&genres=horror&start=101
https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&genres=horror&start=151
https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&genres=horror&start=201
https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&genres=horror&start=251
https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&genres=horror&start=301
https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&genres=horror&start=351
https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&genres=horror&start=401
https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&genres=horror&start=451
https://www.imdb.com/search/title?title_type=feature&num_votes=1000,&g

,horror movie,year,imdb-ratings,metascore,votes,runtime,gross*10^6,certificates
0,Child's Play,2019,6.4,48,8095,90,2.356e+07,R
1,Annabelle Comes Home,2019,6.5,53,7720,106,3.11e+07,R
2,Brightburn,2019,6.4,44,20303,90,1.73e+07,R
3,I Am Mother,2019,6.8,64,31859,113,Null,TV-14
4,Us,2019,7.0,81,118628,116,1.7501e+08,R
5,The Dead Don't Die,2019,6.0,54,6540,104,5.89e+06,R
6,Pet Sematary,2019,5.9,57,40126,101,5.472e+07,R
7,Hereditary,2018,7.3,87,155358,127,4.407e+07,R
8,The Perfection,2018,6.1,60,18021,90,Null,TV-MA
9,The Nun,2018,5.4,46,91618,96,1.1744e+08,R
